In [56]:
# For colab training
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# repo_path = '/content/drive/MyDrive/projects/Celpose'
# if not os.path.exists(repo_path):
#     os.makedirs(repo_path)
# os.chdir(repo_path)

# !pip install cellpose

# Single file API segmentation test

In [57]:
image_input_file = r"D:\Junwoo\Projects\Image_processing_tools\Image_processing_tools\input\cellpose\E3H5_Round1_DAPI_tile2_inference.tif"

In [ ]:
import tifffile
import matplotlib.pyplot as plt

# Read the image
img = tifffile.imread(image_input_file)

# Print the shape of the image
print(f"Image shape: {img.shape}")


In [ ]:
# Specify the layer index to visualize (0-based indexing)
layer_index = 300  # Change this to the specific layer you want to visualize

# Check if the layer index is within bounds
if 0 <= layer_index < img.shape[0]:
    plt.figure(figsize=(6, 6))
    plt.imshow(img[layer_index, :, :], cmap='gray')
    plt.title(f'Layer {layer_index + 1}')
    plt.axis('off')
    plt.show()
else:
    print(f"Invalid layer index: {layer_index}. Please choose a value between 0 and {img.shape[0] - 1}.")


In [49]:
from cellpose import denoise, models, io

In [ ]:

# model set up
denoise_model = denoise.DenoiseModel(gpu=True, model_type='denoise_nuclei',  diam_mean=20.0)
model = models.CellposeModel(gpu=True, model_type = "nuclei" , diam_mean=20.0)


# denoise
imgs_dn = denoise_model.eval(img, batch_size=8,
                            channel_axis=None,
                            # z_axis=0,
                            normalize=True,
                            rescale=None,
                            diameter=20,
                            tile=True,
                            do_3D=True,
                            tile_overlap=0.1,
                            bsize=224)

# segmentation
masks, flows, styles = model.eval(
    x=imgs_dn,  # Your input image array
    batch_size=8,
    do_3D=True,       # Enable 3D segmentation
    diameter=20.0,
    channel_axis = None,
    # anisotropy=2.0    # Adjust anisotropy if needed
)

print("output image shape is: ", masks.shape)

# save the masks as tif file
io.save_masks(
    images=[img],  # List containing the image
    masks=[masks],            # List containing the mask
    flows=[flows],            # List containing the flows
    file_names=['output/Cellpose_output'],  # Base filename for saving
    png=False,                # Disable saving as PNG
    tif=True,                 # Enable saving as TIFF
    channels=[0, 0],           # Channels used in Cellpose
    # ... other parameters if needed ...
)

# print(f"Mask number n saved as 'cp_masks_{img_name}.tif'")




In [ ]:
# Add this before saving to debug
print("Masks shape:", masks.shape)
print("Masks dtype:", masks.dtype)

In [ ]:
# # Save the segmentation output
# io.masks_flows_to_seg(
#     images=[img],  # List containing the image
#     masks=[masks],            # List containing the mask
#     flows=[flows],            # List containing the flows
#     file_names=['Image_processing_tools/output/segmented_output_seg.npy'],  # Base filename for saving
#     diams=30.0,              # Diameter used in Cellpose

# )

# print("Segmentation output saved to 'segmented_output_seg.npy'")

# For Loop training

In [1]:
import os
import glob
import tifffile as tiff

In [2]:
from cellpose import denoise, models, io

In [3]:
# TODO test
from skimage import measure 
import pandas as pd

In [4]:
# Initialize models at the beginning
denoise_model = denoise.DenoiseModel(gpu=True, model_type='denoise_nuclei', diam_mean=19.0)
cellpose_model = models.CellposeModel(gpu=True, model_type='nuclei', diam_mean=19.0)

d:\Junwoo\Conda\envs\cellpose\lib\site-packages\cellpose\resnet_torch.py:271: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename, map_location=de

In [15]:
def denoise_and_segment_image(img, img_name, output_path, denoise_model, cellpose_model):
    """
    Perform denoising and segmentation on a 3D image using Cellpose.

    Parameters:
    - img (numpy.ndarray): Input image array to process.
    - img_name (str): Name of the input image (for logging and saving purposes).
    - output_path (str): Base path to save the output.
    - denoise_model: Pre-initialized denoising model.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    None
    """
    # Display input image information
    print(f"Processing file: {img_name}, shape: {img.shape}")

    # Perform denoising
    imgs_dn = denoise_model.eval(
        img,
        batch_size=4,
        channel_axis=None,
        normalize=True,
        rescale=None,
        diameter=19, # denoise_model.diam_mean
        tile=True,
        do_3D=True,
        tile_overlap=0.1,
        bsize=224
    )

    print("Denoise Down!")

    # Perform segmentation
    masks, flows, styles = cellpose_model.eval(
        x=imgs_dn,
        batch_size=8,
        do_3D=True,
        diameter=cellpose_model.diam_mean,
        channel_axis=None
    )

    print("Mask Segmentation Down!")

    # Export centroids
    props = measure.regionprops_table['centroid', 'area']
    props_dataframe = pd.DataFrame(props)
    c_output_path = os.path.join(output_path, "_centroids.csv")

    props_dataframe.to_csv(c_output_path)

    print("Get Centroids Down!")

    # Save the masks as TIFF files
    io.save_masks(
        images=img,
        masks=masks,
        flows=flows,
        file_names=output_path,
        png=False,
        tif=True,
        channels=[0, 0]
    )

    print("Save Mask Down!")

    




In [16]:
def process_images_in_folder(input_folder, output_folder, denoise_model, cellpose_model):
    """
    Process all TIF images in a folder: denoise, segment, and save the output.

    Parameters:
    - input_folder (str): Path to the folder containing input images.
    - output_folder (str): Path to save the processed output images.
    - denoise_model: Pre-initialized denoising model.
    - cellpose_model: Pre-initialized Cellpose segmentation model.

    Returns:
    None
    """
    
    i = 0

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Find all TIF files in the input folder
    file_paths = glob.glob(os.path.join(input_folder, "*.tif"))

    for file_path in file_paths:
        
        # Read the image
        # print("file_path:", file_path)
        img = tiff.imread(file_path)

        # Get the base file name
        file_name = os.path.basename(file_path)
        # print("file_name:", file_name)

        file_base = os.path.splitext(file_name)[0]
        # print("file_base:", file_base)

        file_base = "cp_masks_" + file_base
        # print("file_base2:", file_base)

        # Define the output path for the current image
        output_path = os.path.join(output_folder, file_base)
        # print("output_path:", output_path)

        # Print the file name
        print(f"Processing image {i}")

        img_name = file_name
        
        # Run the denoise and segmentation function
        denoise_and_segment_image(
            img=img,
            img_name=file_name,
            output_path=output_path,   
            denoise_model=denoise_model,
            cellpose_model=cellpose_model
        )

        i += 1

        print(f"Output of {img_name} saved as: {output_path}")


In [17]:

def process_images_in_folder(input_folder, output_folder, denoise_model, cellpose_model):
    """
    Process all TIF images in a folder: denoise, segment, and save the output.
    """
    i = 0

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    print(f"Output folder created/verified: {output_folder}")

    # Find all TIF files in the input folder
    file_paths = glob.glob(os.path.join(input_folder, "*.tif"))
    print(f"Found {len(file_paths)} TIF files in input folder")

    for file_path in file_paths:
        try:
            # Read the image
            print(f"\nProcessing file: {file_path}")
            img = tiff.imread(file_path)
            print(f"Image shape: {img.shape}")

            # Get the base file name
            file_name = os.path.basename(file_path)
            file_base = os.path.splitext(file_name)[0]
            file_base = "cp_masks_" + file_base
            output_path = os.path.join(output_folder, file_base)

            print(f"Processing image {i}: {file_name}")
            print(f"Output will be saved to: {output_path}")

            # Call the processing function
            print("Calling denoise_and_segment_image...")
            result = denoise_and_segment_image(
                img=img,
                img_name=file_name,
                output_path=output_path,   
                denoise_model=denoise_model,
                cellpose_model=cellpose_model
            )
            print("Finished processing image")

            i += 1
            print(f"Successfully processed [{file_name}]. Output saved as: {output_path}")

        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue

In [18]:
# TODO test

def check_files_in_folder(input_folder, output_folder, denoise_model, cellpose_model):
    """
    Process all TIF images in a folder: denoise, segment, and save the output.
    """
    # Print all contents of the input folder
    print(f"Contents of input folder {input_folder}:")
    for item in os.listdir(input_folder):
        print(f"- {item}")

    # Try different extensions
    tif_files = []
    tif_files.extend(glob.glob(os.path.join(input_folder, "*.tif")))
    tif_files.extend(glob.glob(os.path.join(input_folder, "*.tiff")))
    tif_files.extend(glob.glob(os.path.join(input_folder, "*.TIF")))
    tif_files.extend(glob.glob(os.path.join(input_folder, "*.TIFF")))

    print(f"\nFound {len(tif_files)} TIF files:")
    for file in tif_files:
        print(f"- {file}")

    # Check if the input_folder path is correct
    print(f"\nFull input folder path: {os.path.abspath(input_folder)}")

    if len(tif_files) == 0:
        print("\nNo TIF files found. Please check:")
        print("1. The input folder path is correct")
        print("2. The files have the correct extension (.tif, .tiff, .TIF, or .TIFF)")
        print("3. You have read permissions for the folder")
        return

  

In [20]:
# Example usage
process_images_in_folder(
    input_folder="input/cellpose",
    output_folder="output/cellpose",
    denoise_model=denoise_model,
    cellpose_model=cellpose_model
)

Output folder created/verified: output/cellpose
Found 6 TIF files in input folder

Processing file: input/cellpose\E3H5_Round1_DAPI_tile1_inference.tif
Image shape: (827, 1024, 1024)
Processing image 0: E3H5_Round1_DAPI_tile1_inference.tif
Output will be saved to: output/cellpose\cp_masks_E3H5_Round1_DAPI_tile1_inference
Calling denoise_and_segment_image...
Processing file: E3H5_Round1_DAPI_tile1_inference.tif, shape: (827, 1024, 1024)
Denoise Down!
Mask Segmentation Down!
Error processing input/cellpose\E3H5_Round1_DAPI_tile1_inference.tif: 'function' object is not subscriptable

Processing file: input/cellpose\E3H5_Round1_DAPI_tile2_inference.tif
Image shape: (827, 1024, 1024)
Processing image 0: E3H5_Round1_DAPI_tile2_inference.tif
Output will be saved to: output/cellpose\cp_masks_E3H5_Round1_DAPI_tile2_inference
Calling denoise_and_segment_image...
Processing file: E3H5_Round1_DAPI_tile2_inference.tif, shape: (827, 1024, 1024)
Denoise Down!
Mask Segmentation Down!
Error processing 

KeyboardInterrupt: 